In [20]:
import cPickle as pickle
from bs4 import BeautifulSoup

In [143]:
import nltk
from nltk.stem.porter import *

In [211]:
import numpy as np

In [61]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [193]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))

In [210]:
words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1

0
29976
59952
89928
119904
149880
179856
209832
239808
269784


In [212]:
# Go through dictionary, removing entries that have a value less
# than delta
delta = 5
words_dict = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict :
    word_indices[word] = index
    index += 1

In [215]:
# Determine Term Frequencies
tf_vectors = []
for text in question_texts :
    vector = np.zeros(len(word_indices))
    for word in text :
        if word in word_indices :
            vector[word_indices[word]] += 1
    tf_vectors.append(vector)

In [223]:
# Determine IDFS
num_docs_vector = np.zeros(len(word_indices))
N = float(len(questions))
for text in question_texts :
    temp_dict = {}
    for word in text :
        if word in word_indices :
            if word in temp_dict :
                continue
            else :
                temp_dict[word] = True
                num_docs_vector[word_indices[word]] += 1

idf_vector = np.log(N / num_docs_vector)                

In [232]:
tf_vectors = np.array(tf_vectors)

In [236]:
tfidf_vectors = tf_vectors * idf_vector[None, :]

In [244]:
tfidf_norms = np.linalg.norm(tfidf_vectors, axis=1)

In [245]:
tfidf_norms.shape

(299760,)

In [247]:
example = tfidf_vectors[0]

In [251]:
results = tfidf_vectors.dot(example) / (np.linalg.norm(example) * tfidf_norms)

In [252]:
results

array([ 1.        ,  0.0252973 ,  0.01509236, ...,  0.02882916,
        0.04584923,  0.03577984])

In [253]:
results[0] = 0

In [254]:
results

array([ 0.        ,  0.0252973 ,  0.01509236, ...,  0.02882916,
        0.04584923,  0.03577984])

In [259]:
results[results == 1.0] = 0

In [260]:
np.argmax(results)

10050

In [261]:
results[10050]

0.13626589854042973

In [262]:
questions[10050]

{u'answer_count': 1,
 u'body': u"<p>I am trying to create a custom layout for my blog listing and blog content display pages.</p>\n\n<p>The following url will display all the latest blogs based on the conditions that I have set.</p>\n\n<p><code>http://fqdn/general-blog</code></p>\n\n<p>When I click on the 'Read more' button corresponding to a button, it will display the full blog contents in another page.</p>\n\n<p>Sample format : <code>http://fqdn/general-blog/tourist-details</code></p>\n\n<p>I am able to modify the layout of the page corresponding to blog listing. For this, I have created a custom tpl file : page--general-blog.tpl.php.</p>\n\n<p>What I want is to create a new template file for displaying the contents of individual blogs.</p>\n\n<p>The individual blog content display page is not entering - page--general-blog.tpl.php file. It's always using the template - page.tpl.php.</p>\n\n<p>I tried creating separate tpl files like :</p>\n\n<p><code>page--general-blog--%.tpl.php</c

In [263]:
questions[0]

{u'answer_count': 1,
 u'body': u'<p>The problem I am encountering is strange. Suppose I have:</p>\n\n<pre><code> a = "www.XXXXXXX.com"\n b = "www.XXXXXXX.com/laskdfj/=*&amp;9809f/12-613"\n c = "www.XXXX.comllkjldfjlsadjfjldsf"\n d = "http://www.XXXX.CoMmasldfjl"\n e = "www.XXX.us/sdf"\n f = "www.XXX.us0948klsdf"\n</code></pre>\n\n<p>If following after the ".com" or ".us" is not a slash, then remove it. So the result would be like:</p>\n\n<pre><code> a = "www.XXXXXXX.com"\n b = "www.XXXXXXX.com/laskdfj/=*&amp;9809f/12-613"\n c = "www.XXXX.com"\n d = "http://www.XXXX.CoM"\n e = "www.XXX.us/sdf"\n f = "www.XXX.us"\n</code></pre>\n\n<p>Regular expression is new to me, and I read several blogs about regular expression, none of them seem to talk about how to use if-statement to handle my situation... any hints?</p>\n',
 u'creation_date': 1447570358,
 u'is_answered': True,
 u'last_activity_date': 1447571841,
 u'last_edit_date': 1447571841,
 u'link': u'http://stackoverflow.com/questions/337172